In [14]:
# Import libraries

import pandas as pd
from statsmodels.tsa.api import ARDL
from statsmodels.tsa.ardl import ardl_select_order

In [15]:
data = pd.read_csv('data.csv')

In [18]:
sp1 = ['E-MINI var','E-MINI var','E-MINI var',"FTSE var","FTSE var","FTSE var","TREASURY var","TREASURY var","TREASURY var"]
sp2 = [["FTSE var"],["TREASURY var"],["FTSE var", "TREASURY var"],["TREASURY var"],['E-MINI var'],["TREASURY var",'E-MINI var'],['E-MINI var'],["FTSE var"],['E-MINI var',"FTSE var"]]
param = []
for k in range(len(sp1)):
    sel_res = ardl_select_order(data[sp1[k]], 10, data[sp2[k]], 10, ic="aic", trend="c")
    print(f"The optimal order is: {sel_res.model.ardl_order}")
    param.append(list(sel_res.model.ardl_order))

The optimal order is: (4, 10)
The optimal order is: (4, 8)
The optimal order is: (4, 10, 7)
The optimal order is: (9, 9)
The optimal order is: (9, 8)
The optimal order is: (9, 10, 8)
The optimal order is: (10, 10)
The optimal order is: (10, 10)
The optimal order is: (10, 10, 10)


In [20]:
min_aic = 0
b=0
for i in range(len(param)):
    if len(param[i]) == 3:

        res = ARDL(data[sp1[i]], param[i][0], data[sp2[i]], {sp2[i][0]: param[i][1],sp2[i][1]: param[i][2]}, trend="c").fit()
        check = res.aic
        if check < min_aic:
            min_aic = check
            b=i
    else:

        res = ARDL(data[sp1[i]], param[i][0], data[sp2[i]], {sp2[i][0]: param[i][1]}, trend="c").fit()
        check = res.aic
        if check < min_aic:
            min_aic = check
            b=i
    print('Model №'+str(i) + "  AIC = " + str(check))
if len(sp2[b]) ==2:
    print('Best model: ' + sp1[b] + ' ~ ' + sp2[b][0] + ' + ' + sp2[b][1] + '  AIC = '+str(min_aic) )
else:
    print('Best model: ' + sp1[b] + ' ~ ' + sp2[b][0] + '  AIC = '+str(min_aic) )

Model №0  AIC = -2162644.8196815676
Model №1  AIC = -2110530.4929715046
Model №2  AIC = -2170553.045366913
Model №3  AIC = -8325635.885554879
Model №4  AIC = -8384989.628805071
Model №5  AIC = -8385582.619996358
Model №6  AIC = -6634082.724736119
Model №7  AIC = -6626179.090077225
Model №8  AIC = -6634508.023699978
Best model: FTSE var ~ TREASURY var + E-MINI var  AIC = -8385582.619996358


In [28]:
# Best model

res = ARDL(data[sp1[5]], param[5][0], data[sp2[5]], {sp2[5][0]: param[5][1],sp2[5][1]: param[5][2]}, trend="c").fit()
print(res.summary())

                              ARDL Model Results                              
Dep. Variable:               FTSE var   No. Observations:               428976
Model:                 ARDL(9, 10, 8)   Log Likelihood             4192822.310
Method:               Conditional MLE   S.D. of innovations              0.000
Date:                Tue, 26 Apr 2022   AIC                       -8385582.620
Time:                        21:03:58   BIC                       -8385242.577
Sample:                            10   HQIC                      -8385485.741
                               428976                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -2.314e-08    2.1e-08     -1.101      0.271   -6.43e-08    1.81e-08
FTSE var.L1         -0.0201      0.002    -13.169      0.000      -0.023      -0.017
FTSE var.L2         -0.0039 

In [22]:
# Ручной подбор лагов с помощью AR

import warnings
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings('ignore')
min_aic= 0
p=0
for i in range(1,11):
    model = ARIMA(data['E-MINI var'], order=(i, 0, 0))
    model_fit = model.fit()
    aic = model_fit.aic
    str1 = 'p = '+str(i)+'  AIC: ' + str(aic)
    if aic < min_aic:
        min_aic = aic
        p = i
    print(str1)
print('Наименьший E-MINI AIC: ' + str(min_aic) + "  p = "+str(p))

p = 1  AIC: -2099796.437184755
p = 2  AIC: -2099891.5941642593
p = 3  AIC: -2099891.6758297775
p = 4  AIC: -2099936.165548868
p = 5  AIC: -2099936.823267348
p = 6  AIC: -2099935.733618816
p = 7  AIC: -2099934.6850729445
p = 8  AIC: -2099938.8465663088
p = 9  AIC: -2099938.6463982477
p = 10  AIC: -2099943.088310487
Наименьший AIC: -2099943.088310487  p = 10


In [23]:
min_aic = 0
p=0
for i in range(1,11):
    model = ARIMA(data['FTSE var'], order=(i, 0, 0))
    model_fit = model.fit()
    aic = model_fit.aic
    str1 = 'p = '+str(i)+'  AIC: ' + str(aic)
    if aic < min_aic:
        min_aic = aic
        p = i
    print(str1)
print('Наименьший FTSE AIC: ' + str(min_aic) + "  p= "+str(p))

p = 1  AIC: -8322307.88797823
p = 2  AIC: -8322335.154492384
p = 3  AIC: -8322340.260506065
p = 4  AIC: -8322388.145544998
p = 5  AIC: -8322401.196347332
p = 6  AIC: -8322444.47608064
p = 7  AIC: -8322468.324630486
p = 8  AIC: -8322478.909302002
p = 9  AIC: -8322495.247322613
p = 10  AIC: -8322493.442545184
Наименьший FTSE AIC: -8322495.247322613  p= 9


In [24]:
min_aic = 0
p=0
for i in range(1,11):
    model = ARIMA(data['TREASURY var'], order=(i, 0, 0))
    model_fit = model.fit()
    aic = model_fit.aic
    str1 = 'p = '+str(i)+'  AIC: ' + str(aic)
    if aic < min_aic:
        min_aic = aic
        p = i
    print(str1)
print('Наименьший TREASURY AIC: ' + str(min_aic) + "  p= "+str(p))

p = 1  AIC: -6621587.385934435
p = 2  AIC: -6621977.53549008
p = 3  AIC: -6622491.1277971035
p = 4  AIC: -6622646.115991382
p = 5  AIC: -6622893.389123309
p = 6  AIC: -6622935.046377009
p = 7  AIC: -6622968.697496264
p = 8  AIC: -6622998.434090967
p = 9  AIC: -6623062.462043968
p = 10  AIC: -6623064.7506114235
Наименьший TREASURY AIC: -6623064.7506114235  p= 10


In [25]:
min_aic = 0
b=0
param1 = [[10,9],[10,10],[10,9,10],[9,10],[9,10],[9,10,10],[10,10],[10,9],[10,10,9]]
for i in range(len(param)):
    if len(param[i]) == 3:

        res = ARDL(data[sp1[i]], param1[i][0], data[sp2[i]], {sp2[i][0]: param1[i][1],sp2[i][1]: param1[i][2]}, trend="c").fit()
        check = res.aic
        if check < min_aic:
            min_aic = check
            b=i
    else:

        res = ARDL(data[sp1[i]], param[i][0], data[sp2[i]], {sp2[i][0]: param[i][1]}, trend="c").fit()
        check = res.aic
        if check < min_aic:
            min_aic = check
            b=i
    print('Model №'+str(i) + "  AIC = " + str(check))
if len(sp2[b]) ==2:
    print('Best model: ' + sp1[b] + ' ~ ' + sp2[b][0] + ' + ' + sp2[b][1] + '  AIC = '+str(min_aic) )
else:
    print('Best model: ' + sp1[b] + ' ~ ' + sp2[b][0] + '  AIC = '+str(min_aic) )

Model №0  AIC = -2162644.8196815676
Model №1  AIC = -2110530.4929715046
Model №2  AIC = -2170506.4768531686
Model №3  AIC = -8325635.885554879
Model №4  AIC = -8384989.628805071
Model №5  AIC = -8385578.67055131
Model №6  AIC = -6634082.724736119
Model №7  AIC = -6626179.090077225
Model №8  AIC = -6634498.179647481
Best model: FTSE var ~ TREASURY var + E-MINI var  AIC = -8385578.67055131
